# 1. Load packages

In [1]:
%load_ext autoreload
%autoreload 2

from analysis_tools.common import *
from util import *

import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
import sklearn
import cv2
import os

np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)
sklearn.random.seed(RANDOM_STATE)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


# 2. Load dataset

In [2]:
size        = 512
input_shape = (size, size, 3)
train_full_data_meta, test_data_meta, X_train_full, X_test = load_data(size)

[########################################] | 100% Completed |  9.9s
[########################################] | 100% Completed |  4.7s


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_sample_weight

classes = train_full_data_meta['class'].unique()
X_train_c, X_val_c, y_train_c, y_val_c = {}, {}, {}, {}
y_enc_c = {}
train_ds_c, val_ds_c = {}, {}

for c in tqdm(classes):
    idxs = train_full_data_meta.query(f"`class` == '{c}' and state != 'good'").index
    X_train_full_c = X_train_full[idxs]
    y_train_full_c = train_full_data_meta.loc[idxs, ['label']]
    X_train_c[c], X_val_c[c], y_train_c[c], y_val_c[c] = train_test_split(X_train_full_c, y_train_full_c, stratify=y_train_full_c)
    
    y_enc_c[c]   = OneHotEncoder(sparse=False, dtype=bool)
    y_train_c[c] = y_enc_c[c].fit_transform(y_train_c[c])
    y_val_c[c]   = y_enc_c[c].transform(y_val_c[c])
    
    train_ds_c[c] = preprocess(tf.data.Dataset.from_tensor_slices((X_train_c[c], y_train_c[c], compute_sample_weight(class_weight='balanced', y=y_train_c[c].argmax(1)))), True, BATCH_SIZE)
    val_ds_c[c]   = preprocess(tf.data.Dataset.from_tensor_slices((X_val_c[c], y_val_c[c],     compute_sample_weight(class_weight='balanced', y=y_val_c[c].argmax(1)))), True, BATCH_SIZE)

100%|██████████| 15/15 [00:07<00:00,  1.94it/s]


In [4]:
from analysis_tools.modeling import *

model_c = {}
for c in tqdm(classes):
    print(f"<{c}>")
    model_name = f'experiment1_{c}'
    epochs = 300

    model_c[c], fine_tuning_on = build_model(len(y_enc_c[c].categories_[0]), strategy, input_shape=input_shape)
    model_c[c].fit(train_ds_c[c], validation_data=val_ds_c[c], epochs=epochs, callbacks=get_callbacks(patience=30, plot_path=join(PATH.result, model_name), init_lr=2e-3, epochs=epochs, warmup_epoch=20, min_lr=1e-3))

    fine_tuning_on()
    model_c[c].fit(train_ds_c[c], validation_data=val_ds_c[c], epochs=epochs, callbacks=get_callbacks(patience=30, plot_path=join(PATH.result, model_name), init_lr=2e-4, epochs=epochs, warmup_epoch=20, min_lr=1e-4))

  0%|          | 0/15 [00:00<?, ?it/s]

<transistor>
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:te

Epoch 41/300
learning rate:  0.001925
1/1 [==============================] - 1s 1s/step - loss: 0.3786 - f1_score: 1.0000 - val_loss: 0.8053 - val_f1_score: 0.5000
Epoch 42/300
learning rate:  0.0019214286
1/1 [==============================] - 1s 1s/step - loss: 0.3324 - f1_score: 1.0000 - val_loss: 0.6940 - val_f1_score: 0.6667
Epoch 43/300
learning rate:  0.00191785721
1/1 [==============================] - 1s 1s/step - loss: 0.3161 - f1_score: 1.0000 - val_loss: 0.7077 - val_f1_score: 1.0000
Epoch 44/300
learning rate:  0.00191428582
1/1 [==============================] - 1s 1s/step - loss: 0.3138 - f1_score: 1.0000 - val_loss: 0.7446 - val_f1_score: 0.6667
Epoch 45/300
learning rate:  0.00191071443
1/1 [==============================] - 1s 1s/step - loss: 0.3384 - f1_score: 1.0000 - val_loss: 0.8679 - val_f1_score: 0.3500
Epoch 46/300
learning rate:  0.00190714293
1/1 [==============================] - 1s 1s/step - loss: 0.2898 - f1_score: 1.0000 - val_loss: 0.4634 - val_f1_score:

1/1 [==============================] - 1s 1s/step - loss: 0.1240 - f1_score: 1.0000 - val_loss: 0.5087 - val_f1_score: 1.0000
Epoch 138/300
learning rate:  0.00157857151
1/1 [==============================] - 1s 1s/step - loss: 0.0598 - f1_score: 1.0000 - val_loss: 0.5541 - val_f1_score: 0.6667
Epoch 139/300
learning rate:  0.00157500012
1/1 [==============================] - 1s 1s/step - loss: 0.0922 - f1_score: 1.0000 - val_loss: 0.2569 - val_f1_score: 1.0000
Epoch 140/300
learning rate:  0.00157142861
1/1 [==============================] - 1s 1s/step - loss: 0.1072 - f1_score: 1.0000 - val_loss: 0.3912 - val_f1_score: 1.0000
Epoch 141/300
learning rate:  0.00156785722
1/1 [==============================] - 1s 1s/step - loss: 0.0783 - f1_score: 1.0000 - val_loss: 0.4665 - val_f1_score: 1.0000
Epoch 142/300
learning rate:  0.00156428583
1/1 [==============================] - 1s 1s/step - loss: 0.0702 - f1_score: 1.0000 - val_loss: 0.2924 - val_f1_score: 1.0000
Epoch 143/300
learning r

1/1 [==============================] - 1s 1s/step - loss: 0.0515 - f1_score: 1.0000 - val_loss: 0.4628 - val_f1_score: 1.0000


...

Epoch 00179: early stopping
<grid>
Epoch 1/300
learning rate:  0.0001
1/1 [==============================] - 39s 39s/step - loss: 1.7540 - f1_score: 0.1164 - val_loss: 1.6752 - val_f1_score: 0.1571
Epoch 2/300
learning rate:  0.0002
1/1 [==============================] - 5s 5s/step - loss: 1.7108 - f1_score: 0.0965 - val_loss: 1.6679 - val_f1_score: 0.1571
Epoch 3/300
learning rate:  0.0003
1/1 [==============================] - 2s 2s/step - loss: 1.6926 - f1_score: 0.1504 - val_loss: 1.5817 - val_f1_score: 0.1571
Epoch 4/300
learning rate:  0.0004
1/1 [==============================] - 2s 2s/step - loss: 1.6734 - f1_score: 0.0762 - val_loss: 1.6127 - val_f1_score: 0.1571
Epoch 5/300
learning rate:  0.0005
1/1 [==============================] - 2s 2s/step - loss: 1.6306 - f1_score: 0.1352 - val_loss: 1.5919 - val_f1_score: 0.1571
Epoch 6/300
learning rate:  0.0006
1/1 [==============================] - 1s 1s/step - loss: 1.6374 - f1_score: 0.1175 - val_loss: 1.5851 - val_f1_score: 0.15

Epoch 50/300
learning rate:  0.00189285725
1/1 [==============================] - 2s 2s/step - loss: 0.9505 - f1_score: 0.6145 - val_loss: 1.5816 - val_f1_score: 0.5000
Epoch 51/300
learning rate:  0.00188928563
1/1 [==============================] - 2s 2s/step - loss: 1.1375 - f1_score: 0.5689 - val_loss: 1.4848 - val_f1_score: 0.3800
Epoch 52/300
learning rate:  0.00188571436
1/1 [==============================] - 2s 2s/step - loss: 0.8220 - f1_score: 0.7244 - val_loss: 1.3410 - val_f1_score: 0.3133
Epoch 53/300
learning rate:  0.00188214285
1/1 [==============================] - 2s 2s/step - loss: 0.8995 - f1_score: 0.7232 - val_loss: 1.2975 - val_f1_score: 0.3000
Epoch 54/300
learning rate:  0.00187857146
1/1 [==============================] - 2s 2s/step - loss: 0.8835 - f1_score: 0.6845 - val_loss: 1.4734 - val_f1_score: 0.3333
Epoch 55/300
learning rate:  0.00187500007
1/1 [==============================] - 2s 2s/step - loss: 0.8976 - f1_score: 0.7348 - val_loss: 1.3812 - val_f1_

1/1 [==============================] - 2s 2s/step - loss: 0.5510 - f1_score: 0.8600 - val_loss: 1.2109 - val_f1_score: 0.3000
Epoch 39/300
learning rate:  0.000193214277
1/1 [==============================] - 2s 2s/step - loss: 0.5036 - f1_score: 0.9556 - val_loss: 1.3137 - val_f1_score: 0.5000
Epoch 40/300
learning rate:  0.000192857129
1/1 [==============================] - 2s 2s/step - loss: 0.6881 - f1_score: 0.9492 - val_loss: 1.4344 - val_f1_score: 0.5000
Epoch 41/300
learning rate:  0.000192499981
1/1 [==============================] - 2s 2s/step - loss: 0.5864 - f1_score: 0.8592 - val_loss: 1.4760 - val_f1_score: 0.3800
Epoch 42/300
learning rate:  0.000192142848
1/1 [==============================] - 2s 2s/step - loss: 0.5722 - f1_score: 0.8529 - val_loss: 1.3042 - val_f1_score: 0.6667
Epoch 43/300
learning rate:  0.000191785715
1/1 [==============================] - 2s 2s/step - loss: 0.4900 - f1_score: 0.8692 - val_loss: 1.5719 - val_f1_score: 0.4667
Epoch 44/300
learning ra

100%|██████████| 15/15 [2:31:04<00:00, 604.29s/it]

Epoch 00044: early stopping
